The ImageNet dataset has more than 14 million images, hand-labeled across 20,000 categories

The downside – is that its too much for an everyday laptop. So the alternative solution: That’s where Fast.ai‘s Imagenette dataset comes in.
> Download this to the current folder: https://s3.amazonaws.com/fast-ai-imageclas/imagenette2.tgz

## Data

In [1]:
imagenette_map = { 
    "n01440764" : "tench",
    "n02102040" : "springer",
    "n02979186" : "casette_player",
    "n03000684" : "chain_saw",
    "n03028079" : "church",
    "n03394916" : "French_horn",
    "n03417042" : "garbage_truck",
    "n03425413" : "gas_pump",
    "n03445777" : "golf_ball",
    "n03888257" : "parachute"
}

In [2]:
from keras.preprocessing.image import ImageDataGenerator

In [3]:
imagegen = ImageDataGenerator()

In [4]:
train = imagegen.flow_from_directory("imagenette2/train/", class_mode = "categorical", shuffle = False, batch_size = 128, target_size = (224, 224))
val = imagegen.flow_from_directory("imagenette2/val/", class_mode = "categorical", shuffle = False, batch_size = 128, target_size = (224, 224))

Found 9469 images belonging to 10 classes.
Found 3925 images belonging to 10 classes.


## Basic CNN model for image classification

In [5]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dense, InputLayer, BatchNormalization, Dropout

In [6]:
model = Sequential()

In [7]:
model.add(InputLayer(input_shape = (224, 224, 3)))

In [8]:
# 1st convolutional block
model.add(Conv2D(25, (5, 5), activation = 'relu', strides = (1, 1), padding = 'same'))
model.add(MaxPool2D(pool_size = (2, 2), padding = 'same'))

In [9]:
# 2nd convolutional block
model.add(Conv2D(50, (5, 5), activation = 'relu', strides = (2, 2), padding = 'same'))
model.add(MaxPool2D(pool_size = (2, 2), padding = 'same'))
model.add(BatchNormalization())

In [10]:
# 3rd convolutional block
model.add(Conv2D(70, (3, 3), activation = 'relu', strides = (2, 2), padding = 'same'))
model.add(MaxPool2D(pool_size = (2, 2), padding = 'valid'))
model.add(BatchNormalization())

In [11]:
# ANN (Artificial Neural Networks) block
model.add(Flatten())
model.add(Dense(units = 100, activation = 'relu'))
model.add(Dense(units = 100, activation = 'relu'))
model.add(Dropout(0.25))

In [12]:
# Output layer
model.add(Dense(units = 10, activation = 'softmax'))

In [13]:
# Compile the model
model.compile(loss = 'categorical_crossentropy', metrics = ['accuracy'], optimizer = "adam")
# deep architecture than what we have utilized so far, we are only able to get a validation accuracy of around 40-50%.

In [14]:
# Fit for N epochs
N = 20
model.fit_generator(train, epochs = N, validation_data = val)

C:\Users\Lidor\AppData\Local\Temp\ipykernel_4780\2699342898.py:3: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train, epochs = N, validation_data = val)


Epoch 1/20
74/74 [==============================] - 234s 3s/step - loss: 2.5344 - accuracy: 0.1393 - val_loss: 3.0824 - val_accuracy: 0.1141
Epoch 2/20
74/74 [==============================] - 223s 3s/step - loss: 2.1076 - accuracy: 0.2675 - val_loss: 2.1511 - val_accuracy: 0.2739
Epoch 3/20
74/74 [==============================] - 266s 4s/step - loss: 1.8631 - accuracy: 0.3665 - val_loss: 1.8768 - val_accuracy: 0.3625
Epoch 4/20
74/74 [==============================] - 240s 3s/step - loss: 1.5537 - accuracy: 0.4800 - val_loss: 1.9371 - val_accuracy: 0.3564
Epoch 5/20
74/74 [==============================] - 227s 3s/step - loss: 1.3432 - accuracy: 0.5519 - val_loss: 3.4085 - val_accuracy: 0.1391
Epoch 6/20
74/74 [==============================] - 226s 3s/step - loss: 1.1910 - accuracy: 0.5956 - val_loss: 2.1495 - val_accuracy: 0.3518
Epoch 7/20
74/74 [==============================] - 229s 3s/step - loss: 0.9949 - accuracy: 0.6626 - val_loss: 2.1247 - val_accuracy: 0.3625
Epoch 8/20
74

#### Transfer learning (VGG16) 

In [18]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.utils import to_categorical

In [16]:
pretrained_model = VGG16(include_top = False, weights = 'imagenet')
pretrained_model.summary()

58889256/58889256 [==============================] - 3s 0us/step
Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, None, None, 3)]   0         
                                                                 
 block1_conv1 (Conv2D)       (None, None, None, 64)    1792      
                                                                 
 block1_conv2 (Conv2D)       (None, None, None, 64)    36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, None, None, 64)    0         
                                                                 
 block2_conv1 (Conv2D)       (None, None, None, 128)   73856     
                                                                 
 block2_conv2 (Conv2D)       (None, None, None, 128)   147584    
                                                              

In [17]:
vgg_features_train = pretrained_model.predict(train)
vgg_features_val = pretrained_model.predict(val)

31/31 [==============================] - 519s 17s/step


In [19]:
train_target = to_categorical(train.labels)
val_target = to_categorical(val.labels)

In [20]:
model2 = Sequential()
model2.add(Flatten(input_shape=(7, 7, 512)))
model2.add(Dense(100, activation = 'relu'))
model2.add(Dropout(0.5))
model2.add(BatchNormalization())
model2.add(Dense(10, activation = 'softmax'))

In [21]:
model2.compile(optimizer = 'adam', metrics = ['accuracy'], loss = 'categorical_crossentropy')

In [22]:
model2.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_1 (Flatten)         (None, 25088)             0         
                                                                 
 dense_3 (Dense)             (None, 100)               2508900   
                                                                 
 dropout_1 (Dropout)         (None, 100)               0         
                                                                 
 batch_normalization_2 (Bat  (None, 100)               400       
 chNormalization)                                                
                                                                 
 dense_4 (Dense)             (None, 10)                1010      
                                                                 
Total params: 2510310 (9.58 MB)
Trainable params: 2510110 (9.58 MB)
Non-trainable params: 200 (800.00 Byte)
____________

In [24]:
N = 40
model2.fit(vgg_features_train, train_target, epochs = N, batch_size = 128, validation_data = (vgg_features_val, val_target))

Epoch 1/40
74/74 [==============================] - 3s 41ms/step - loss: 0.0920 - accuracy: 0.9837 - val_loss: 0.1782 - val_accuracy: 0.9450
Epoch 2/40
74/74 [==============================] - 3s 37ms/step - loss: 0.0475 - accuracy: 0.9948 - val_loss: 0.1731 - val_accuracy: 0.9460
Epoch 3/40
74/74 [==============================] - 3s 36ms/step - loss: 0.0273 - accuracy: 0.9979 - val_loss: 0.1721 - val_accuracy: 0.9457
Epoch 4/40
74/74 [==============================] - 3s 39ms/step - loss: 0.0193 - accuracy: 0.9990 - val_loss: 0.1746 - val_accuracy: 0.9462
Epoch 5/40
74/74 [==============================] - 3s 37ms/step - loss: 0.0138 - accuracy: 0.9996 - val_loss: 0.1789 - val_accuracy: 0.9460
Epoch 6/40
74/74 [==============================] - 3s 37ms/step - loss: 0.0130 - accuracy: 0.9993 - val_loss: 0.1762 - val_accuracy: 0.9450
Epoch 7/40
74/74 [==============================] - 3s 36ms/step - loss: 0.0093 - accuracy: 0.9996 - val_loss: 0.1792 - val_accuracy: 0.9457
Epoch 8/40
74

Future steps, need to try the same on the following datasubsets
 - Imagewoof: 10 classes of dog breeds – a larger batch size and a more difficult problem to classify.
 - Image网 (“wang”): A combination of Imagenette and Imagewoof and a couple of tricks that make it a harder problem.